# what is this?
Experimneting with data augmentation for NLP. The goal is to experiment with the dataaugmentation to increase training size of teh original data.

## install required libraries

In [ ]:
import random

In [ ]:
!pip install fastprogress

## import all

In [ ]:
import pandas as pd
import nltk
import spacy
import multiprocessing as mp
from typing import List
from functools import partial
from spacy.lang.en import English
from nltk.corpus import wordnet as wn
from pandas.core.frame import DataFrame
from fastai.text.all import *
from fastprogress import progress_bar
from fastcore.all import *

## Getting the data

In [ ]:
df_train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
df_train.head(5)

use wordnet to repalce random word with their synonyms

In [ ]:
nltk.download('wordnet')

## Creating helpers
creating helpers to find synonyms and replace them with picked words

quick check to see how the internals works

In [ ]:
synsets = wn.synsets("leave", pos="v")
lemmas = synsets[0].lemmas()
lemmas[1].name()

In [ ]:
spacy_doc = "i am a cat"
" ".join([spacy_doc[:3], "awesome", spacy_doc[1 + 2 :]])

In [ ]:
text = "As Roger had predicted, the snow departed"
L(text.split())

In [ ]:
wn.synsets?

In [ ]:
text2 = "Apple is looking at buying U.K. startup for $1 billion"
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
for token in doc:
    print(token.text, token.pos_)

In [ ]:
def tokenify(text: str):
    """Split text into a list with optional rules"""
    #text = text.lower()
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    return {tok.text:tok.pos_ for tok in doc}
    #return L(tok.text for tok in doc)
    
def get_synonyms(token, pos=None):
    """Get the synonyms of word from Wordnet."""
    lemmas = set().union(*[s.lemmas() for s in wn.synsets(token, pos=pos)])
    return list(set(l.name().lower().replace("_", " ") for l in lemmas) - {token})   

In [ ]:
tokenify(text)

creating transformation functions to replace adjectives, nouns, verbs with synonyms

In [ ]:
text.index("predicted")

## Write transformation functions

### Replace with synonym

In [ ]:
def replace_tok_with_synonym(text: str, pos="v"):
    """replace a random verb with a synonym."""
    pos_dict = {"v": "VERB", "a": "ADJ", "n": "Noun"}
    tokens = tokenify(text)
    pos_words = L(tok for tok in tokens.keys() if tokens[tok] == pos_dict[pos])
    idx = random.choice(range(len(pos_words))) if pos_words else None
    synonyms = get_synonyms(pos_words[idx], pos=pos) if idx else None
    if synonyms:
        return text.replace(pos_words[idx], synonyms[0])
    else:
        return None

### Random deletion
Code taken directly from the paper and the related repo [here](https://github.com/jasonwei20/eda_nlp/blob/master/code/eda.py)

In [ ]:
a = {"a":1, "b":2}
list(a.keys())

In [ ]:
def random_deletion(text: str, p: float= 0.2):
    """Delete random tokens from a text"""
    tokens = list(tokenify(text).keys())
    if len(tokens) == 1: return tokens
    new_tokens = []
    
    for token in tokens:
        r = random.uniform(0, 1)
        if r > p:
            new_tokens.append(token)

    #if you end up deleting all words, just return a random word
    if len(new_tokens) == 0:
        rand_int = random.randint(0, len(tokens)-1)
        return [tokens[rand_int]]

    return ' '.join(new_tokens)

def erase(p: float= 0.2):
    return partial(random_deletion, p = p)

### Random swap

In [ ]:
def swap(tokens: List, threshold: int):
    """swap two tokens"""
    idx1 = random.randint(0, len(tokens)-1)
    idx2 = idx1
    counter = 0
    while idx2 == idx1:
        idx2 = random.randint(0, len(tokens)-1)
        counter += 1
        if counter > threshold:
            return tokens
        tokens[idx1], tokens[idx2] =  tokens[idx2], tokens[idx1]
        
    return tokens

def random_swap(text: str,
                threshold: int= 3):
    """swap random tokens in a text
    and return the transformed text"""
    tokens = list(tokenify(text).keys())
    n_times = max(1, len(tokens))
    for _ in range(n_times):
        tokens = swap(tokens, threshold)
    return ' '.join(tokens)

def swapabble(threshold:int = 3):
    return partial(random_swap,
                   threshold=threshold)


In [ ]:
random_swap("hello there, how are you")

quick test

In [ ]:
sample_text = df_train.excerpt.loc[0]
sample_text

In [ ]:
replace_tok_with_synonym(sample_text)

In [ ]:
replace_tok_with_synonym(sample_text, "n")

In [ ]:
df_tiny = df_train.loc[:3]
df_tiny

In [ ]:
#df_tiny["excerpt"] = df_tiny["excerpt"].transform(replace_tok_with_synonym,"n")

In [ ]:
df_tiny

In [ ]:
random_deletion("i am a superhero, but...", 0.2)

In [ ]:
#this gives error
#erase("i am a superhero, but...")

In [ ]:
erase = erase()
erase("i am a superhero, but...")

creating separate functions for verbs adjectives and nouns

In [ ]:
def replace_verb(text: str):
    """replace verb with it's synonym"""
    return replace_tok_with_synonym(text)

def replace_noun(text: str):
    """replace noun with it's synonym"""
    return replace_tok_with_synonym(text, "n")

def replace_adj(text: str):
    """replace adjective with it's synonym"""
    return replace_tok_with_synonym(text, "a")

In [ ]:
replace_verb("I predicted this would happen sometime")

quick test

In [ ]:
df_tiny2 = df_train.loc[:1]
df_tiny2

# Designing transform fucntion
* Should apply functions to the dataframe one by one
* After applying each function it should concatenate the transformed dataframe with the original and then return a new dataframe.
  * While doing so, it should remove the rows for which the text field was returned none

In [ ]:
def transform(df: DataFrame, col:str, f):
    df_tf = df.copy()
    df_tf[col] = df_tf[col].transform(f)
    df_tf.dropna(inplace=True, subset=['excerpt'])
    return df_tf

In [ ]:
sample_df = df_train.loc[:5]
sample_df.shape

In [ ]:
sample1= transform(df_tiny2, 'excerpt', replace_verb)

In [ ]:
sample2 = transform(df_tiny2, 'excerpt', partial(random_deletion,p=0.2))

In [ ]:
sample3 = transform(df_tiny2, 'excerpt', erase)

In [ ]:
sample4 = transform(df_tiny2, 'excerpt', swapabble(threshold=3))

In [ ]:
sample1.shape, sample2.shape, sample3.shape, sample4.shape

In [ ]:
sample1

In [ ]:
sample2

In [ ]:
sample3

In [ ]:
sample4

## Multiprocessing
Parallelly run all the diffeerent varient of the transform function i.e. with different parameters and then concatenate all results.

list of parameter variants

In [ ]:
erase = erase
swapabble = swapabble()
args = [replace_verb, 
        replace_noun, 
        replace_adj,
        erase,
       swapabble]

collection callback

In [ ]:
results = []
def collect_result(result):
    results.append(result)

In [ ]:
n_workers=mp.cpu_count()
pause=0

creating partialtransform

In [ ]:
tfm = partial(transform, sample_df, 'excerpt')

parallel processing with single function

In [ ]:
#result[0].shape

In [ ]:
#result[0].head()

parallel processing with multiple functions

In [ ]:
#with pool(n_workers) as ex:
#    result2 = ex.map(tfm, [replace_verb, replace_noun, replace_adj])

In [ ]:
#len(result2)

In [ ]:
#result2

## The augmenter
* Take the transform function and run parallely on multiple replace functions.
* Collect results and do post cleanup.
* Put in a single dataframe

In [ ]:
progress_bar?

In [ ]:
def augmenter(df: DataFrame, col: str, tmflist: List) -> DataFrame:
    n_workers=mp.cpu_count()
    partial_tmf = partial(transform, df, col)
    r = parallel(partial_tmf, tmflist, progress=True)
    return pd.concat([df]+r, ignore_index=True)

In [ ]:
sample_df.shape

In [ ]:
final_df = augmenter(sample_df, 'excerpt', args)

In [ ]:
final_df

In [ ]:
final_df.shape

# Augmenting the full dataset
Generating excerpts by replacing noun, verb and adjective with their synonyms if any.

In [ ]:
aug_df = augmenter(df_train, 'excerpt', args)

checking the augmented dataframe's shape

In [ ]:
aug_df.shape

checking a couple of data

In [ ]:
aug_df.head(2)

In [ ]:
aug_df.tail(3)

# Saving the augmented dataframe

In [ ]:
aug_df.to_csv('./aug_df.csv')